Assignment 6 - Indexing and Materialized Views

## indexes

indexes are a type of data structure, that allows faster lookups in databases. An index is created for a certain column/key, and this index will contain a subset of the table data. When you lookup rows based on the indexed column, the index is traversed instead of the table itself. This allows for a more optimized lookup, and will make the query execute faster (since the indexed data is usually ordered).

Indexing comes with a cost though, and too heavy use of it can result in slower insert, update and delete queries.

## join queries

Join queries combines multiple tables into sets, typically based on foreign keys.

In [ ]:
# %load_ext sql

In [9]:
%sql postgresql://appdev@data2/appdev

'Connected: appdev@appdev'

In [12]:
%sql select * from pg_indexes where tablename = 'circuits';

3 rows affected.


schemaname,tablename,indexname,tablespace,indexdef
f1db,circuits,idx_17102_url,None,CREATE UNIQUE INDEX idx_17102_url ON circuits USING btree (url)
f1db,circuits,circuits_position_idx,None,"CREATE INDEX circuits_position_idx ON circuits USING gist (""position"")"
f1db,circuits,idx_17102_primary,None,CREATE UNIQUE INDEX idx_17102_primary ON circuits USING btree (circuitid)


As we can see, the 'circuits' table has 3 indexes:
- 'url' is indexed, using the default binary tree type. Url has a string value and so is easily sorted, making the binary tree an appropriate choice.
- 'position' is indexed using gist, the generalised search tree. An R-tree could also have been used, but R-trees are especially useful when you want to index coordinates based on the distance between them, which makes it great for looking up coordinates within a certain area. Gist is the more general choice and was probably sufficient in this case.
- 'circuitid' is also indexed with the binary tree type, since it is a number type and also easily sorted.

In [53]:
%sql SELECT * from circuits where circuits.circuitid = 1

1 rows affected.


circuitid,circuitref,name,location,country,lat,lng,alt,url,position
1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,http://en.wikipedia.org/wiki/Melbourne_Grand_Prix_Circuit,"(144.968,-37.8497)"


In [64]:
%%sql 
SELECT 
    races.date, driver.surname, EXTRACT(YEAR from AGE(CURRENT_DATE, driver.dob)) as "age", 
    results.milliseconds, races.name, circuits.name
FROM 
    (select * from drivers where dob > '03-19-1980') as driver
INNER JOIN
    (select * from results where position = 1 ) as results 
    on driver.driverid = results.driverid 
LEFT JOIN
    races 
    on results.raceid = races.raceid
LEFT JOIN
    circuits
    on races.circuitid = circuits.circuitid

184 rows affected.


date,surname,age,milliseconds,name,name_1
2008-03-16,Hamilton,33.0,5690616,Australian Grand Prix,Albert Park Grand Prix Circuit
2008-04-06,Massa,36.0,5466970,Bahrain Grand Prix,Bahrain International Circuit
2008-05-11,Massa,36.0,5209451,Turkish Grand Prix,Istanbul Park
2008-05-25,Hamilton,33.0,7242742,Monaco Grand Prix,Circuit de Monaco
2008-06-08,Kubica,33.0,5784227,Canadian Grand Prix,Circuit Gilles Villeneuve
2008-06-22,Massa,36.0,5510245,French Grand Prix,Circuit de Nevers Magny-Cours
2008-07-06,Hamilton,33.0,5949440,British Grand Prix,Silverstone Circuit
2008-07-20,Hamilton,33.0,5480874,German Grand Prix,Hockenheimring
2008-08-03,Kovalainen,36.0,5847067,Hungarian Grand Prix,Hungaroring
2008-08-24,Massa,36.0,5732339,European Grand Prix,Valencia Street Circuit


In [61]:
%%sql 
EXPLAIN ANALYZE
SELECT 
    races.date, driver.surname, driver.dob, results.milliseconds, 
    races.name, circuits.name
FROM 
    (select * from drivers where dob > '03-19-1980') as driver
INNER JOIN
    (select * from results where position = 1 ) as results 
    on driver.driverid = results.driverid 
LEFT JOIN
    races 
    on results.raceid = races.raceid
LEFT JOIN
    circuits
    on races.circuitid = circuits.circuitid

18 rows affected.


QUERY PLAN
Nested Loop Left Join (cost=25.67..784.78 rows=68 width=62) (actual time=0.522..11.861 rows=184 loops=1)
-> Nested Loop Left Join (cost=25.53..772.81 rows=68 width=50) (actual time=0.492..10.411 rows=184 loops=1)
-> Hash Join (cost=25.25..746.50 rows=68 width=27) (actual time=0.423..8.203 rows=184 loops=1)
Hash Cond: (results.driverid = drivers.driverid)
-> Seq Scan on results (cost=0.00..708.96 rows=974 width=24) (actual time=0.028..7.242 rows=974 loops=1)
"Filter: (""position"" = 1)"
Rows Removed by Filter: 22703
-> Hash (cost=24.51..24.51 rows=59 width=19) (actual time=0.376..0.376 rows=60 loops=1)
Buckets: 1024 Batches: 1 Memory Usage: 12kB
-> Seq Scan on drivers (cost=0.00..24.51 rows=59 width=19) (actual time=0.014..0.338 rows=60 loops=1)


## the query

The analysis displays 16 calls/operations. I tried to optimize the query by filtering with WHERE before joining the tables. This means the WHERE is executed with the least amount of elements to evaluate possible. The query is executed in just above 12 ms, which is decently fast. To further optimize, one could look at the order of the joins, since you could possibly get better performance this way. It is also interesting to note that the analysis tells us about how the indexes affect the query speed. This is also a point where one could look for opportunities to achieve faster queries.

In [67]:
%%sql CREATE MATERIALIZED VIEW race_winners_cache AS
SELECT 
    races.date, driver.surname, driver.dob, results.milliseconds, 
    races.name as "race_name", circuits.name as "circuit_name"
FROM 
    (select * from drivers where dob > '03-19-1980') as driver
INNER JOIN
    (select * from results where position = 1 ) as results 
    on driver.driverid = results.driverid 
LEFT JOIN
    races 
    on results.raceid = races.raceid
LEFT JOIN
    circuits
    on races.circuitid = circuits.circuitid

184 rows affected.


[]

In [68]:
%sql EXPLAIN ANALYSE SELECT * FROM race_winners_cache

3 rows affected.


QUERY PLAN
Seq Scan on race_winners_cache (cost=0.00..10.50 rows=50 width=1564) (actual time=0.018..0.066 rows=184 loops=1)
Planning time: 0.229 ms
Execution time: 0.120 ms


Creating a materialised view cached the results of the query, allowing us to access the result of the query extremely quickly. The drawback is that the query will not be updated when data is changed. When this happens we need to manually refresh the view. Materialised Views could be useful in situations where you do not need the newest data, for example when viewing querying for data over long periods of time for statistical analysis for instance. You could refresh the data at regular intervals, perhaps during low-traffic hours of the day.